# Parsing  

In [24]:
#Author: Chetana
import nltk
from nltk.tokenize import word_tokenize
import string

# Context Free Grammar

In [29]:
grammar = nltk.CFG.fromstring("""
    S -> NP VP
    NP -> Det NP | Adj NP | Adj | N | Adv NP
    VP -> V NP  
    Det -> "a" | "an" | "the" 
    V -> "is"
    Adj -> "interesting" | "very"
    N -> "NLP" | "subject"
    Adv -> "very"
""")

statement = "NLP is very interesting"
sentence = word_tokenize(statement)
print(sentence)
print(nltk.pos_tag(sentence))

['NLP', 'is', 'very', 'interesting']
[('NLP', 'NNP'), ('is', 'VBZ'), ('very', 'RB'), ('interesting', 'JJ')]


In [30]:
# Recursive descent parser is a kind of top-down parser 
# built from a set of mutually recursive procedures 
# where each such procedure implements one of the nonterminals of the grammar. 
rd_parser = nltk.RecursiveDescentParser(grammar)
total_trees = 0
for tree in rd_parser.parse(sentence):
    total_trees = total_trees+1
    print(tree)
    tree.draw()

(S (NP (N NLP)) (VP (V is) (NP (Adj very) (NP (Adj interesting)))))
(S (NP (N NLP)) (VP (V is) (NP (Adv very) (NP (Adj interesting)))))


In [31]:
if total_trees > 1 :
    print("Ambiguious grammar")
else:
    print("Unambiguious grammar")    

Ambiguious grammar


# Chart Parser

In [32]:
# When a chart parser begins parsing a text, it creates a new (empty) chart, spanning the text. 
# It then incrementally adds new edges to the chart.  
# A set of "chart rules" specifies the conditions under which new edges should be added to the chart.
# Once the chart reaches a stage where none of the chart rules adds any new edges, parsing is complete.
grammar1 = nltk.CFG.fromstring("""
    S -> NP VP 
    VP -> V NP | Aux VP | V
    NP -> Det NP | N | Adj NP | Adj
    N -> "girl" | "boy"
    Det -> "The"
    Aux -> "is"
    V -> "laughing" | "playing" 
    Adj -> "laughing" | "well"
""")
statement = nltk.word_tokenize("The girl is laughing")
print(nltk.pos_tag(statement))

[('The', 'DT'), ('girl', 'NN'), ('is', 'VBZ'), ('laughing', 'VBG')]


In [33]:
total_trees = 0
rd_parser = nltk.ChartParser(grammar1)
for tree in rd_parser.parse(statement):
    total_trees = total_trees + 1
    print(tree)
    tree.draw()

(S (NP (Det The) (NP (N girl))) (VP (Aux is) (VP (V laughing))))


In [7]:
if total_trees > 1 :
    print("Ambiguious grammar")
else:
    print("Unambiguious grammar")

Unambiguious grammar


In [34]:
grammar1 = nltk.CFG.fromstring("""
    S -> NP VP 
    VP -> V NP | Aux VP | V NP PP | V
    PP -> P NP
    NP -> Det NP | N | Adj NP | Adj | Det N PP
    N -> "girl" | "boy" | "Omkar" | "can" | "hold" | "water"
    Det -> "The" | "a" | "the"
    Aux -> "is" | "can"
    P -> "of" | "with"
    V -> "laughing" | "playing" | "can" | "hold" | "water"
    Adj -> "laughing" | "well"
""")
statement = nltk.word_tokenize("The can can hold a can of water")
print(nltk.pos_tag(statement))

[('The', 'DT'), ('can', 'MD'), ('can', 'MD'), ('hold', 'VB'), ('a', 'DT'), ('can', 'MD'), ('of', 'IN'), ('water', 'NN')]


In [35]:
tree_count = 0
chart_parser = nltk.ChartParser(grammar1)
for tree in chart_parser.parse(statement):
    tree_count = tree_count+1
    print(tree)
    tree.draw()

(S
  (NP (Det The) (NP (N can)))
  (VP
    (Aux can)
    (VP
      (V hold)
      (NP (Det a) (NP (N can)))
      (PP (P of) (NP (N water))))))
(S
  (NP (Det The) (NP (N can)))
  (VP
    (Aux can)
    (VP (V hold) (NP (Det a) (N can) (PP (P of) (NP (N water)))))))


In [10]:
if tree_count > 1 :
    print("Ambiguos Sentence")
else :
    print("Unambiguos Sentence")

Ambiguos Sentence


# Probabilitic Context Free Grammar (PCFG)
A PCFG consists of a start state and a set of productions with probabilities. The set of terminals and nonterminals is implicitly specified by the productions.

# Inside Chart Parser

In [36]:
parser = pchart.InsideChartParser(grammar)
times=[]
print('\n sentence: %s\n parser: %s\n grammar_rules: %s' % (sent,parser,grammar))
parser.trace(1)

ValueError: The grammar must be probabilistic PCFG

In [18]:
t = time.time()
parses = parser.parse_all(tokens)
times.append(time.time()-t)
print("the time required by the Inside Chart parser  %s "%(times))

  |. . . . . . [-]| [6:7] 'telescope'                [1.0]
  |. . . . . [-] .| [5:6] 'the'                      [1.0]
  |. . . . [-] . .| [4:5] 'with'                     [1.0]
  |. . . [-] . . .| [3:4] 'man'                      [1.0]
  |. . [-] . . . .| [2:3] 'the'                      [1.0]
  |. [-] . . . . .| [1:2] 'saw'                      [1.0]
  |[-] . . . . . .| [0:1] 'John'                     [1.0]
  |. . [-] . . . .| [2:3] Det -> 'the' *             [0.8]
  |. . > . . . . .| [2:2] Det -> * 'the'             [0.8]
  |. . . . . [-] .| [5:6] Det -> 'the' *             [0.8]
  |. . . . . > . .| [5:5] Det -> * 'the'             [0.8]
  |. [-] . . . . .| [1:2] V  -> 'saw' *              [0.65]
  |. > . . . . . .| [1:1] VP -> * V NP               [0.7]
  |. > . . . . . .| [1:1] V  -> * 'saw'              [0.65]
  |. . . . [-] . .| [4:5] P  -> 'with' *             [0.61]
  |. . . . > . . .| [4:4] PP -> * P NP               [1.0]
  |. . . . [-> . .| [4:5] PP -> P * NP               

In [37]:
for parse in parses:
    print(parse)

(S
  (NP John)
  (VP
    (V saw)
    (NP
      (NP (Det the) (N man))
      (PP (P with) (NP (Det the) (N telescope)))))) (p=0.00027755)


# Viterbi Parser

In [19]:
# PCFG parser that uses dynamic programming to find the single most likely parse 
# for a text. The ViterbiParser parser parses texts by filling in a “most likely constituent table”.
# This table records the most probable tree representation for any given span and node value.

from nltk import ViterbiParser
demos = [('John saw the man with the telescope' , toy_pcfg1)]

sent, grammar = demos[0]

# Tokenize the sentence.
tokens = word_tokenize(sent)
print(tokens)
print(nltk.pos_tag(tokens))

['John', 'saw', 'the', 'man', 'with', 'the', 'telescope']
[('John', 'NNP'), ('saw', 'VBD'), ('the', 'DT'), ('man', 'NN'), ('with', 'IN'), ('the', 'DT'), ('telescope', 'NN')]


In [38]:
parser = ViterbiParser(grammar)
times=[]
print('\n sentence: %s\n parser: %s\n grammar_rules: %s' % (sent,parser,grammar))


 sentence: John saw the man with the telescope
 parser: <ViterbiParser for <Grammar with 16 productions>>
 grammar_rules: Grammar with 16 productions (start state = S)
    S -> NP VP
    NP -> Det NP
    NP -> Adj NP
    NP -> Adj
    NP -> N
    NP -> Adv NP
    VP -> V NP
    Det -> 'a'
    Det -> 'an'
    Det -> 'the'
    V -> 'is'
    Adj -> 'interesting'
    Adj -> 'very'
    N -> 'NLP'
    N -> 'subject'
    Adv -> 'very'


In [21]:
parser.trace(1)
t = time.time()
parses = parser.parse_all(tokens)
times.append(time.time()-t)
print("the time required by the Viterbi parser  %s "%(times))

Inserting tokens into the most likely constituents table...
Finding the most likely constituents spanning 1 text elements...
Finding the most likely constituents spanning 2 text elements...
Finding the most likely constituents spanning 3 text elements...
Finding the most likely constituents spanning 4 text elements...
Finding the most likely constituents spanning 5 text elements...
Finding the most likely constituents spanning 6 text elements...
Finding the most likely constituents spanning 7 text elements...
the time required by the Viterbi parser  [0.0069789886474609375] 


In [22]:
for parse in parses:
    print(parse)

(S
  (NP John)
  (VP
    (V saw)
    (NP
      (NP (Det the) (N man))
      (PP (P with) (NP (Det the) (N telescope)))))) (p=0.00027755)
